In [1]:
# import necessary libraries
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge, Lasso
from sklearn.linear_model import RidgeCV, LassoCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error

### 1. Handling Missing Value and Type of Dataset

In [2]:
# load datasets
train_data = pd.read_csv('datasets/train_data.csv')
test_data = pd.read_csv('datasets/test.csv', keep_default_na=False, na_values="") 

ref_test = test_data
# set the max columns to none
pd.set_option('display.max_columns', None)

In [3]:
# check the data
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1954 entries, 0 to 1953
Data columns (total 26 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Overall Qual        1954 non-null   float64
 1   Exter Qual          1954 non-null   float64
 2   Bsmt Qual           1954 non-null   float64
 3   1st Flr SF          1954 non-null   int64  
 4   Gr Liv Area         1954 non-null   int64  
 5   Kitchen Qual        1954 non-null   float64
 6   Fireplace Qu        1954 non-null   float64
 7   Garage Finish       1954 non-null   float64
 8   Garage Area         1954 non-null   float64
 9   Mas Vnr Avail       1954 non-null   int64  
 10  House_Style_1Story  1954 non-null   int64  
 11  House_Style_2Story  1954 non-null   int64  
 12  House_Style_Others  1954 non-null   int64  
 13  Foundation__CBlock  1954 non-null   int64  
 14  Foundation__Others  1954 non-null   int64  
 15  Foundation__PConc   1954 non-null   int64  
 16  Area_S

In [4]:
fts_keep = train_data.columns.tolist()

fts_keep.remove('SalePrice')
fts_keep

['Overall Qual',
 'Exter Qual',
 'Bsmt Qual',
 '1st Flr SF',
 'Gr Liv Area',
 'Kitchen Qual',
 'Fireplace Qu',
 'Garage Finish',
 'Garage Area',
 'Mas Vnr Avail',
 'House_Style_1Story',
 'House_Style_2Story',
 'House_Style_Others',
 'Foundation__CBlock',
 'Foundation__Others',
 'Foundation__PConc',
 'Area_Side_East',
 'Area_Side_North',
 'Area_Side_South',
 'Area_Side_West',
 'Total_Bath',
 'Age',
 'Age_Renov',
 'Bsmt_QC',
 'Area_Rm_Ratio']

In [5]:
test_data.head()

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Type,Mas Vnr Area,Exter Qual,Exter Cond,Foundation,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Heating,Heating QC,Central Air,Electrical,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Fireplace Qu,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type
0,2658,902301120,190,RM,69.0,9142,Pave,Grvl,Reg,Lvl,AllPub,Inside,Gtl,OldTown,Norm,Norm,2fmCon,2Story,6,8,1910,1950,Gable,CompShg,AsbShng,AsbShng,None,0.0,TA,Fa,Stone,Fa,TA,No,Unf,0,Unf,0,1020,1020,GasA,Gd,N,FuseP,908,1020,0,1928,0,0,2,0,4,2,Fa,9,Typ,0,NA,Detchd,1910.0,Unf,1,440,Po,Po,Y,0,60,112,0,0,0,NA,NA,NA,0,4,2006,WD
1,2718,905108090,90,RL,NaN,9662,Pave,NA,IR1,Lvl,AllPub,Inside,Gtl,Sawyer,Norm,Norm,Duplex,1Story,5,4,1977,1977,Gable,CompShg,Plywood,Plywood,None,0.0,TA,TA,CBlock,Gd,TA,No,Unf,0,Unf,0,1967,1967,GasA,TA,Y,SBrkr,1967,0,0,1967,0,0,2,0,6,2,TA,10,Typ,0,NA,Attchd,1977.0,Fin,2,580,TA,TA,Y,170,0,0,0,0,0,NA,NA,NA,0,8,2006,WD
2,2414,528218130,60,RL,58.0,17104,Pave,NA,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,7,5,2006,2006,Gable,CompShg,VinylSd,VinylSd,None,0.0,Gd,TA,PConc,Gd,Gd,Av,GLQ,554,Unf,0,100,654,GasA,Ex,Y,SBrkr,664,832,0,1496,1,0,2,1,3,1,Gd,7,Typ,1,Gd,Attchd,2006.0,RFn,2,426,TA,TA,Y,100,24,0,0,0,0,NA,NA,NA,0,9,2006,New
3,1989,902207150,30,RM,60.0,8520,Pave,NA,Reg,Lvl,AllPub,Inside,Gtl,OldTown,Norm,Norm,1Fam,1Story,5,6,1923,2006,Gable,CompShg,Wd Sdng,Wd Sdng,None,0.0,Gd,TA,CBlock,TA,TA,No,Unf,0,Unf,0,968,968,GasA,TA,Y,SBrkr,968,0,0,968,0,0,1,0,2,1,TA,5,Typ,0,NA,Detchd,1935.0,Unf,2,480,Fa,TA,N,0,0,184,0,0,0,NA,NA,NA,0,7,2007,WD
4,625,535105100,20,RL,NaN,9500,Pave,NA,IR1,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,5,1963,1963,Gable,CompShg,Plywood,Plywood,BrkFace,247.0,TA,TA,CBlock,Gd,TA,No,BLQ,609,Unf,0,785,1394,GasA,Gd,Y,SBrkr,1394,0,0,1394,1,0,1,1,3,1,TA,6,Typ,2,Gd,Attchd,1963.0,RFn,2,514,TA,TA,Y,0,76,0,0,185,0,NA,NA,NA,0,7,2009,WD


In [6]:
test_data.shape

(878, 80)

In [7]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 878 entries, 0 to 877
Data columns (total 80 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Id               878 non-null    int64  
 1   PID              878 non-null    int64  
 2   MS SubClass      878 non-null    int64  
 3   MS Zoning        878 non-null    object 
 4   Lot Frontage     718 non-null    float64
 5   Lot Area         878 non-null    int64  
 6   Street           878 non-null    object 
 7   Alley            878 non-null    object 
 8   Lot Shape        878 non-null    object 
 9   Land Contour     878 non-null    object 
 10  Utilities        878 non-null    object 
 11  Lot Config       878 non-null    object 
 12  Land Slope       878 non-null    object 
 13  Neighborhood     878 non-null    object 
 14  Condition 1      878 non-null    object 
 15  Condition 2      878 non-null    object 
 16  Bldg Type        878 non-null    object 
 17  House Style     

From the info of the null value, the columns that we need to fill out the missing value is Mas Vnr Type

In [8]:
test_data[test_data['Mas Vnr Type'].isnull()]

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Type,Mas Vnr Area,Exter Qual,Exter Cond,Foundation,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Heating,Heating QC,Central Air,Electrical,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Fireplace Qu,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type
865,868,907260030,60,RL,70.0,8749,Pave,NA,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2002,2002,Gable,CompShg,VinylSd,VinylSd,NaN,NaN,Gd,TA,PConc,Gd,TA,No,Unf,0,Unf,0,840,840,GasA,Ex,Y,SBrkr,840,885,0,1725,0,0,2,1,3,1,Gd,6,Typ,0,NA,Attchd,2002.0,RFn,2,550,TA,TA,Y,0,48,0,0,0,0,NA,NA,NA,0,11,2009,WD


In [9]:
# we assume the property does not masonry veneer type

# Function to change Null value to string 'NA'
def NulltoNone(value):
    if pd.isna(value):
        return 'None'
    else:
        return value

test_data['Mas Vnr Type'] = test_data['Mas Vnr Type'].apply(NulltoNone)

In [10]:
test_data[test_data['Garage Finish'].isnull()]

,Id,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Type,Mas Vnr Area,Exter Qual,Exter Cond,Foundation,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Heating,Heating QC,Central Air,Electrical,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Fireplace Qu,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type
764,1357,903426160,60,RM,57.0,8094,Pave,Grvl,Reg,Lvl,AllPub,Inside,Gtl,OldTown,Norm,Norm,2fmCon,2.5Unf,6,8,1910,1983,Gable,CompShg,Wd Sdng,Wd Sdng,None,0.0,TA,TA,PConc,TA,TA,Mn,Rec,196,Unf,0,1046,1242,GasA,Gd,Y,SBrkr,1242,742,0,1984,0,0,2,0,5,1,TA,8,Typ,0,NA,Detchd,NaN,NaN,1,360,NA,NA,Y,64,0,180,0,0,0,NA,MnPrv,Shed,1000,9,2008,WD


In [11]:
# Function to change Null value to string 'NA'
def NulltoNA(value):
    if pd.isna(value):
        return 'NA'
    else:
        return value
test_data['Garage Finish'] = test_data['Garage Finish'].apply(NulltoNA)

### 2. Recreating the train_data Dataset Features

In [12]:
# House Style dictionary into 3 categories: 1 Story, 2 Story and Others
hstyle_dict = {'1.5Unf':'Others', '1.5Fin':'Others', 'SFoyer':'Others', '1Story':'1Story', 
               '2.5Unf':'Others', 'SLvl':'Others', '2Story':'2Story', '2.5Fin':'Others'}

# train_data['House_Style'] = train_data['House Style'].apply(lambda hstyle: 'MultiStory' if hstyle != '1Story' else hstyle)
test_data['House_Style'] = test_data['House Style'].map(hstyle_dict)


In [13]:
# create Foundation_ column to store 3 categories: CBlock, PConc and Others
test_data['Foundation_'] = test_data['Foundation'].apply(lambda f: f if (f == 'CBlock' or f == 'PConc') else 'Others')

In [14]:
# categorise the Mas Vnr Type into two categories None and Avail
test_data['Mas Vnr Avail'] = test_data['Mas Vnr Type'].apply(lambda mvtype: 'NA' if mvtype == 'None' else 'Avail')

# binarize the Mas Vnr Avail
test_data['Mas Vnr Avail'] = test_data['Mas Vnr Avail'].map({'NA':0, 'Avail':1})

In [15]:
# dictionary of the neighborhood with its cardinal direction of Iowa State University
nghbrhd_dict = {'NAmes': 'North', 'CollgCr': 'West', 'OldTown': 'East', 'Edwards': 'West', 'Somerst': 'North', 'Gilbert': 'North',
                'Sawyer': 'West', 'SawyerW': 'West', 'NWAmes': 'North', 'Mitchel': 'South', 'NridgHt': 'North', 'BrkSide': 'East',
                'Crawfor': 'South', 'IDOTRR': 'East', 'Timber': 'South', 'NoRidge': 'North', 'SWISU': 'South', 'ClearCr': 'West',
                'MeadowV': 'South', 'Blmgntn': 'North', 'StoneBr': 'North', 'BrDale': 'North', 'NPkVill': 'North', 'Veenker': 'North',
                'Blueste': 'South', 'Greens': 'North', 'GrnHill': 'South', 'Landmrk': 'West'}

# categorise the Neigborhood by the cardinal direction which the center is the IOWA State University
test_data['Area_Side'] = test_data['Neighborhood'].map(nghbrhd_dict)

In [16]:
dummies = ['House_Style', 'Foundation_', 'Area_Side']

test_data = pd.get_dummies(test_data, prefix= dummies, columns= dummies)

In [17]:
# list of ordinal features
ordinal_fts = ['Overall Qual','Exter Qual', 'Bsmt Qual', 'Kitchen Qual', 
               'Garage Finish', 'Bsmt Cond', 'Fireplace Qu']


In [18]:
# dictionary of the ordinal features with each attributes
ordinal_dict = {'Lot Shape':['IR3', 'IR2', 'IR1', 'Reg'],
                'Utilities':['ELO','NoSeWa','NoSewr','AllPub'],
                'Land Slope':['Sev','Mod','Gtl'],
                'Overall Qual':[0,1,2,3,4,5,6,7,8,9,10], 
                'Overall Cond':[0,1,2,3,4,5,6,7,8,9,10], 
                'Exter Qual':['Po', 'Fa', 'TA', 'Gd', 'Ex'], 
                'Exter Cond':['Po', 'Fa', 'TA', 'Gd', 'Ex'], 
                'Bsmt Qual':['NA', 'Po', 'Fa', 'TA', 'Gd', 'Ex'], 
                'Bsmt Cond':['NA', 'Po', 'Fa', 'TA', 'Gd', 'Ex'], 
                'Bsmt Exposure':['NA', 'No', 'Mn', 'Av', 'Gd'], 
                'BsmtFin Type 1':['NA', 'Unf', 'LwQ', 'Rec', 'BLQ', 'ALQ', 'GLQ'], 
                'BsmtFin Type 2':['NA', 'Unf', 'LwQ', 'Rec', 'BLQ', 'ALQ', 'GLQ'], 
                'Heating QC':['Po', 'Fa', 'TA', 'Gd', 'Ex'], 
                'Electrical':['Mix','FuseP','FuseF','FuseA','SBrkr'], 
                'Kitchen Qual':['Po', 'Fa', 'TA', 'Gd', 'Ex'], 
                'Functional':['Sal', 'Sev', 'Maj2', 'Maj1', 'Mod', 'Min2', 'Min1', 'Typ'], 
                'Fireplace Qu':['NA', 'Po', 'Fa', 'TA', 'Gd', 'Ex'], 
                'Garage Finish':['NA','Unf','RFn','Fin'], 
                'Garage Qual':['NA', 'Po', 'Fa', 'TA', 'Gd', 'Ex'], 
                'Garage Cond':['NA', 'Po', 'Fa', 'TA', 'Gd', 'Ex'], 
                'Paved Drive':['N', 'P', 'Y'],
                'Pool QC':['NA','Fa','TA','Gd','Ex'],
                'Fence':['NA','MnWw','GdWo','MnPrv','GdPrv']}

In [19]:
# function for mapping the column values with a numeric scale
def MaptoScale(value, vlist):
    return vlist.index(value)

# iterate through all the ordinal columns list to apply the function and change data type to float
for column in ordinal_fts:
    test_data[column] = test_data[column].apply(MaptoScale, vlist = ordinal_dict[column]).astype('float')

In [20]:
# create new feature of Total_Bath
test_data = test_data.assign(Total_Bath= lambda x: x['Bsmt Full Bath'] + (0.5*x['Bsmt Half Bath']) + x['Full Bath'] + (0.5*x['Half Bath']))

# create new feature of Age
test_data = test_data.assign(Age= lambda x: x['Yr Sold'] - x['Year Built'])

# create new feature of Age_Renov
test_data = test_data.assign(Age_Renov= lambda x: x['Yr Sold'] - x['Year Remod/Add'])

# create new feature of Bsmt_QC
test_data = test_data.assign(Bsmt_QC= lambda x: x['Total Bsmt SF'] * x['Bsmt Cond'])

# create new feature of Area_Rm_Ratio
test_data = test_data.assign(Area_Rm_Ratio= lambda x: (x['Gr Liv Area'] / x['TotRms AbvGrd']).round(2))


In [21]:
test_data = test_data[fts_keep]
test_data

,Overall Qual,Exter Qual,Bsmt Qual,1st Flr SF,Gr Liv Area,Kitchen Qual,Fireplace Qu,Garage Finish,Garage Area,Mas Vnr Avail,House_Style_1Story,House_Style_2Story,House_Style_Others,Foundation__CBlock,Foundation__Others,Foundation__PConc,Area_Side_East,Area_Side_North,Area_Side_South,Area_Side_West,Total_Bath,Age,Age_Renov,Bsmt_QC,Area_Rm_Ratio
0,6.0,2.0,2.0,908,1928,1.0,0.0,1.0,440,0,0,1,0,0,1,0,1,0,0,0,2.0,96,56,3060.0,214.22
1,5.0,2.0,4.0,1967,1967,2.0,0.0,3.0,580,0,1,0,0,1,0,0,0,0,0,1,2.0,29,29,5901.0,196.70
2,7.0,3.0,4.0,664,1496,3.0,4.0,2.0,426,0,0,1,0,0,0,1,0,1,0,0,3.5,0,0,2616.0,213.71
3,5.0,3.0,3.0,968,968,2.0,0.0,1.0,480,0,1,0,0,1,0,0,1,0,0,0,1.0,84,1,2904.0,193.60
4,6.0,2.0,4.0,1394,1394,2.0,4.0,2.0,514,1,1,0,0,1,0,0,0,1,0,0,2.5,46,46,4182.0,232.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
873,6.0,2.0,3.0,1084,1877,2.0,3.0,1.0,488,0,0,1,0,1,0,0,0,1,0,0,3.5,33,33,3252.0,234.62
874,6.0,3.0,3.0,1104,1988,3.0,4.0,2.0,480,1,0,1,0,1,0,0,0,1,0,0,2.5,42,9,3312.0,220.89
875,5.0,2.0,3.0,1211,1211,2.0,3.0,1.0,322,0,1,0,0,1,0,0,0,0,0,1,1.0,40,40,2856.0,242.20
876,4.0,2.0,3.0,864,864,2.0,0.0,1.0,528,0,1,0,0,0,0,1,0,1,0,0,1.0,36,36,2592.0,172.80


### 3. Modelling

In [22]:
# create feature matrix and target vector

# load all the selected features exluding the SalePrice
features = [col for col in train_data.columns if col != 'SalePrice']
X_train = train_data[features]
y_train = train_data['SalePrice']

X_test = test_data

In [23]:
# Scaling using Standard Scaler
ss = StandardScaler()
ss.fit(X_train)
X_train = ss.transform(X_train)
X_test = ss.transform(X_test)

In [24]:
# initiating the ridge regression model
# alpha = optimal alpha found using train dataset
opt_alpha_ridge = 2.3816855519761586

ridge = Ridge(alpha= opt_alpha_ridge, random_state= 42)

In [25]:
# fitting the model
ridge.fit(X_train, y_train)

Ridge(alpha=2.3816855519761586, random_state=42)

In [26]:
# obtain the prediction
ridge_pred = ridge.predict(X_test)

In [27]:
ref_test['SalePrice'] = ridge_pred
submit = ref_test[['Id', 'SalePrice']]

In [28]:
submit = submit.sort_values(by= 'Id')

In [29]:
submit

,Id,SalePrice
703,2,117584.514425
705,4,265211.353218
119,6,192882.595675
311,7,219711.371150
400,17,189507.414954
...,...,...
536,2919,86837.008394
178,2921,97622.395187
828,2922,188226.685168
668,2925,161114.042024


In [30]:
# save as csv file
submit.to_csv('datasets/pred_submit.csv', index=False)